# HW 3

## Imports

In [2]:
import nbtools
nbtools.setup_nb()

In [3]:
import itertools
import sympy
from sympy.diffgeom import Manifold, Patch
from pystein import coords, metric, curvature
from pystein.utilities import tensor_pow as tpow, full_simplify

## Problem 1

In [68]:
M = Manifold('M', dim=3)
P = Patch('origin', M)
rho, z, phi = sympy.symbols('rho z phi', nonnegative=True)

psi = sympy.Function('psi')(rho, z, phi)
q = sympy.Function('q')(rho, z)
cs = coords.CoordSystem('cylindrical', P, [rho, z, phi])
drho, dz, dphi = cs.base_oneforms()


ds2 =  (sympy.exp(q) * (tpow(drho, 2) + tpow(dz, 2)) 
                  + rho ** 2 * tpow(dphi, 2))
gamma = metric.Metric(twoform=ds2)

gamma

rho**2*TensorProduct(dphi, dphi) + exp(q(rho, z))*(TensorProduct(drho, drho) + TensorProduct(dz, dz))

In [69]:
ricci = curvature.ricci_scalar(gamma).doit()
ricci = full_simplify(ricci)
ricci

-(Derivative(q(rho, z), (rho, 2)) + Derivative(q(rho, z), (z, 2)))*exp(-q(rho, z))

In [70]:
christoffels = [((i, i, k), curvature.christoffel_symbol_component(k, i, i, gamma).doit()) 
                for i, k in itertools.product(range(3), range(3))]

In [71]:
curvature.display_components(christoffels)

LHS

In [72]:
one_form = [sympy.Derivative(psi, cs.base_symbols()[k]) for k in range(3)]

In [73]:
def cov_deriv_one_form(i, j, form, g, scale_by_inverse: bool = False):
    coord_syms = g.coord_system.base_symbols()
    v = sympy.Derivative(form[j], coord_syms[i])
    
    for k in range(len(coord_syms)):
        term = curvature.christoffel_symbol_component(k, i, j, g) * form[k]
        if scale_by_inverse:
            term *= g.matrix.inverse()[i, j]
        v -= term

    return full_simplify(v.doit())

In [74]:
def Dbar2(i, j, conformal_func, g):
    one_form = [sympy.Derivative(psi, g.coord_system.base_symbols()[k]) for k in range(3)]
    raw = cov_deriv_one_form(i, j, one_form, g)
    return full_simplify(raw.doit())

In [75]:
DiDjPsi = sympy.Matrix([[Dbar2(0, j, psi, gamma) for j in range(3)],
                            [Dbar2(1, j, psi, gamma) for j in range(3)],
                            [Dbar2(2, j, psi, gamma) for j in range(3)]])

In [76]:
Dbar2Psi = sum([gamma.matrix.inverse()[i, j] * DiDjPsi[i, j] 
                for i, j in itertools.product(range(3), range(3))])

In [77]:
LHS = full_simplify(Dbar2Psi)
RHS = psi / 8 * ricci

In [78]:
notation_subs = [
    (psi, sympy.symbols('psi'))
]

In [79]:
LHS.subs(notation_subs)

(rho**2*(Derivative(psi, (rho, 2)) + Derivative(psi, (z, 2))) + rho*Derivative(psi, rho) + exp(q(rho, z))*Derivative(psi, (phi, 2)))*exp(-q(rho, z))/rho**2

In [80]:
RHS.subs(notation_subs)

-psi*(Derivative(q(rho, z), (rho, 2)) + Derivative(q(rho, z), (z, 2)))*exp(-q(rho, z))/8

In [50]:
def flat_laplacian(func):
    return ((1 / rho) * sympy.Derivative((rho * sympy.Derivative(func, rho)), rho)
             + (1 / rho) ** 2 * sympy.Derivative(sympy.Derivative(func, phi), phi)
             + sympy.Derivative(sympy.Derivative(func, z), z))

In [67]:
flat_laplacian(psi).doit().subs(notation_subs)

Derivative(psi, (z, 2)) + (rho*Derivative(psi, (rho, 2)) + Derivative(psi, rho))/rho + Derivative(psi, (phi, 2))/rho**2

In [81]:
gamma.matrix.inverse()

Matrix([
[exp(-q(rho, z)),               0,         0],
[              0, exp(-q(rho, z)),         0],
[              0,               0, rho**(-2)]])

In [ ]:
cov_deriv_one_form()